In [ ]:
%load_ext autoreload
%autoreload 2
import sys, os
import pandas as pd
import matplotlib.pyplot as plt
plt.rcParams.update({
    "text.usetex": True,                # Enable LaTeX text rendering
    "font.family": "serif",             # Use a serif font
    "font.serif": ["Computer Modern"],  # Set font to Computer Modern (LaTeX default)
})

sys.path.append('../src/')
from Biologging_Toolkit.applications.Density import Density
from Biologging_Toolkit.processing.Dives import Dives

In [ ]:
depids = ['ml17_281a', 'ml17_301a', 'ml18_292a', 'ml18_293a', 'ml18_294a', 'ml18_294b', 'ml19_292a',
          'ml19_293a', 'ml19_295a', 'ml19_295c', 'ml19_296a', 'ml20_292a', 'ml20_293a', 'ml20_295a',
          'ml20_296a', 'ml20_296b', 'ml20_296c', 'ml20_304a', 'ml20_313a', 'ml21_282a', 'ml21_295a',
          'ml21_297a', 'ml21_298a', 'ml21_303a', 'ml21_305b']
depid = 'ml19_290b'
path = 'D:/individus_brut/CTD/'
sens_path = os.path.join(path, depid)

### Initiate class

In [ ]:
results = {}
for depid in depids :
    sens_path = os.path.join(path, depid)
    inst = Dives(depid, path = sens_path)
    if len(inst.ds.variables.keys()) == 0:
        continue
    if 'dives' not in inst.ds.__dir__():
        inst()
    if list(inst.ds.dimensions.keys())[0] !=  'time' :
        inst.ds.renameDimension('index', 'time')
    inst.ds.close()
    inst = Density(depid,
            path = sens_path)
    inst.ds.sampling_rate = 1/3
    if list(inst.ds.dimensions.keys())[0] !=  'time' :
        inst.ds.renameDimension('index', 'time')
    inst.get_descent_speed()
    df = pd.DataFrame({'time_Udesc':inst.time_Udesc, 'Udesc':inst.Udesc})
    if list(inst.ds.dimensions.keys())[0] !=  'time' :
        inst.ds.renameDimension('index', 'time')
    inst.get_drift_speed(method = 'acceleration', sens_path = os.path.join(sens_path, depid+'sens5.nc'))
    inst.ds.close()
    df = pd.DataFrame({'time_Udesc':inst.time_Udesc, 'Udesc':inst.Udesc, 'time_Udrift':inst.time_Udrift, 'Udrift':inst.Udrift})
    results = {**results, **{depid:df}}


In [ ]:
results[list(results.keys())[2]]

In [ ]:
import numpy as np
np.savez("inertial_acc.npz", **results)


In [ ]:
test = np.load("max_descent_speed.npz")

In [ ]:
test['ml17_281a']

In [ ]:
results = test

In [ ]:
import matplotlib.ticker as ticker

SECONDS_PER_DAY = 86400
for key in results.keys() :
    fig, ax = plt.subplots()
    time_days = (results[key]['time_Udesc'] - results[key]['time_Udesc'][0]) / SECONDS_PER_DAY
    ax.scatter(time_days, results[key]['Udesc'], alpha = 0.1)
    ax.xaxis.set_major_locator(ticker.MaxNLocator(integer=True))
    ax.set_title(key)
    ax.set_ylim(0, 12)
    fig.savefig(key + '.png')

In [ ]:
import netCDF4 as nc
ds = nc.Dataset(path + depids[16] + '/' + depids[16] + '_sens.nc')

In [ ]:
plt.scatter()

In [ ]:
plt.scatter(results[depids[7]]['time_Udesc'][:], results[depids[7]]['Udesc'][:])

In [ ]:
import scipy
fig, ax = plt.subplots(3, 1, figsize = (6, 12))
for depid in depids[1:] :
    try :
        df = results[depid].copy()
        df['Udrift'] = - df['Udrift'].to_numpy()
        df['med_Udesc'] = scipy.signal.medfilt(df['Udesc'], kernel_size = 5)
        df['day_Udesc'] = pd.to_datetime(df['time_Udesc'], unit = 's').dt.date
        df['day_Udrift'] = pd.to_datetime(df['time_Udrift'], unit = 's').dt.date
        udesc_long = (
            df[["day_Udesc", "med_Udesc"]]
            .dropna(subset=["day_Udesc"])
            .rename(columns={"day_Udesc": "day"})
        )

        udrift_long = (
            df[["day_Udrift", "Udrift"]]
            .dropna(subset=["day_Udrift"])
            .rename(columns={"day_Udrift": "day"})
        )
        udesc_avg = udesc_long.groupby("day", as_index=False)["med_Udesc"].mean()
        udrift_avg = udrift_long.groupby("day", as_index=False)["Udrift"].mean()
        final_df = (pd.merge(udesc_avg, udrift_avg, on="day", how="outer").sort_values("day").reset_index(drop=True))
        ratio = final_df['med_Udesc'].to_numpy() / final_df['Udrift'].to_numpy()
        mask = (ratio > 1) & (ratio < 40)
        ax[0].scatter((final_df.day - final_df.day.iloc[0]).apply(lambda x : x.days),
                      final_df.med_Udesc, alpha = 0.3)
        ax[1].scatter((final_df.day - final_df.day.iloc[0]).apply(lambda x : x.days),
              final_df.Udrift, alpha = 0.3)
        ax[2].scatter((final_df.day - final_df.day.iloc[0]).apply(lambda x : x.days)[mask],
                    ratio[mask], alpha = 0.7, c = 'r')
    except:
        continue
ax[0].set_ylim(0, 15)
ax[0].set_ylabel(r'$U_{z}^{max}$ during descent')
ax[1].set_ylabel(r'$U_z^{mean}$ during drift phases')
ax[2].set_ylabel(r'$U_{z}^{max}$ / $U_z^{mean}$')
ax[2].set_xlabel(r'Day')

In [ ]:
fig.savefig('first_results_density_proxy.pdf')

In [ ]:
plt.show()

In [ ]:
help(datetime.datetime.fromtimestamp)